In [173]:
# !pip install pandas


import pandas as pd
import numpy as np
import json

# load environment variables
# !pip install python-dotenv
# !pip install -U langchain-community
from dotenv import load_dotenv
load_dotenv()
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

# !pip install langchain_pinecone
from langchain_pinecone import PineconeVectorStore, PineconeEmbeddings

from langchain.document_loaders import TextLoader

In [174]:
# read the data "01 課程大綱_202408201909.json" into panda file

with open('課程大綱.json') as f:
    data = json.load(f)

df = pd.DataFrame(data)

# course = df['course']
course = list(data.values())[0]

course = pd.json_normalize(course)


In [175]:
print(course.columns)

Index(['開課學期', '開課學校', '課程狀態', '課程流水號', '課號', '課程識別碼', '班次', '課程名稱', '學分',
       '為必選修', '為預分配', '通識領域', '授課對象系所代碼', '授課對象系所名稱', '授課教師', '職稱', '所屬學院',
       '所屬系所', 'E-mail', '辦公室', '聯絡電話', '個人資訊', '個人網站', '密集課程週次', '上課星期',
       '上課節次', '上課地點', '加簽方式', '修課總人數', '本校人數', '外校人數', '外系人數', '領域專長代碼',
       '領域專長名稱', '為英語授課', 'NTU COOL', '核心能力與課程規劃關聯圖', '課程與基本素養關聯圖', '下載課程大綱檔案',
       '課程備註', '課程概述', '課程目標', '課程要求', '預計課後學習時數', 'OfficeHour需預約',
       'OfficeHour備註', 'OfficeHour星期', 'OfficeHour開始時間', 'OfficeHour結束時間',
       '指定閱讀', '參考書目'],
      dtype='object')


In [189]:
# print all  所屬學院
print(course['所屬學院'].unique())

['文學院' '共同教育中心' None '理學院' '生命科學院' '工學院' '法律學院' '生物資源暨農學院' '醫學院' '管理學院'
 '電機資訊學院' '學程' '社會科學院' '公共衛生學院' '創新設計學院' '重點科技研究學院' '進修推廣學院' 'default']


In [191]:
# print all 所屬系所
print(course['所屬系所'].unique())

['中國文學系' '日本語文學系' '外國語文學系' '共同教育組' None '應用數學科學研究所' '基因體與系統生物學學位學程' '物理學系'
 '地質科學系' '土木工程學系' '法律學系' '生物環境系統工程學系' '藥學系' '財務金融學系' '環境工程學研究所'
 '華語教學碩士學位學程' '醫學檢驗暨生物技術學系' '物理治療學系' '農業經濟學系' '臨床醫學研究所' '植物病理與微生物學系'
 '海洋研究所' '歷史學系' '電機工程學系' '化學工程學系' '圖書資訊學系' '大氣科學系' '生物機電工程學系' '中等學校教育學程'
 '工程科學及海洋工程學系' '天文物理研究所' '植物醫學碩士學位學程' '園藝暨景觀學系' '政治學系' '化學系' '資訊工程學系'
 '機械工程學系' '工商管理學系' '哲學系' '森林環境暨資源學系' '昆蟲學系' '翻譯碩士學位學程' '臨床牙醫學研究所'
 '生物科技研究所' '戲劇學系' '商學研究所' '臨床藥學研究所' '資訊管理學系' '數學系' '國際企業學系' '應用力學研究所'
 '生態學與演化生物學研究所' '國際三校農業生技與健康醫療碩士學位學程' '藝術史研究所' '臨床動物醫學研究所' '經濟學系'
 '創業創新管理碩士在職專班' '免疫學研究所' '光電工程學研究所' '腫瘤醫學研究所' '分子醫學研究所' '職能治療學系'
 '動物科學技術學系' '生理學研究所' '社會工作學系' '地理環境資源學系' '公共衛生學系' '漁業科學研究所' '生化科學研究所'
 '人類學系' '電子工程學研究所' '心理學系' '全球衛生碩士學位學程' '管理學院企業管理碩士專班(GMBA)' '生醫電子與資訊學研究所'
 '材料科學與工程學系' '農業化學系' '生物多樣性國際碩士學位學程' '電信工程學研究所' '應用物理學研究所' '會計學系' '醫學工程學系'
 '分子暨比較病理生物學研究所' '醫療器材與醫學影像研究所' '統計與數據科學研究所' '國際體育運動事務學士學位學程' '建築與城鄉研究所'
 '微生物學研究所' '科際整合法律學研究所' '音樂學研究所' '護理學系' '牙醫學系' '流行病學與預防醫學研究所'
 '資訊網路與多媒體研究所' '食品科技研究所' '生化

In [176]:
# print the szize of the data
print(course.shape)

(83547, 51)


In [177]:
#  filter 開課學期 = 113-1
course = course[course['開課學期'] == '113-1']


In [178]:
print(course.shape)

(17343, 51)


In [179]:
# 只保留
# "課程名稱"
# "課程流水號"
# "所屬系所"
# "授課教師"
# "上課星期"
# "上課節次"
# "課程概述"
# "課程目標"
# "課程要求"
# 學分
# 所屬學院
# 上課地點
# 欄位

course = course[["課程名稱", "課程流水號", "所屬系所", "授課教師", "上課星期", "上課節次", "課程概述", "課程目標", "課程要求", "學分", "所屬學院", "上課地點", "通識領域"]]

In [180]:
print(course.shape)

(17343, 13)


In [181]:
# if "課程名稱" contain "專題討論"， drop the row
course = course[~course["課程名稱"].str.contains("專題")]

In [182]:
print(course.shape)

(12556, 13)


In [183]:
# if "課程概述" or "課程目標" or "課程要求" is empty, remove the row
course = course.dropna(subset=["課程概述", "課程目標", "課程要求"])

In [184]:
course['上課節次'] = course['上課節次'].apply(lambda x: list(filter(str.isalnum, str(x).upper())))

In [185]:
bool_check = [0,0,0,0,0,0,0,0]

for idx, c in course.iterrows():
    if c['通識領域'] is None:
        continue
    
    # Remove braces if present and split by comma
    domains = c['通識領域'].strip("{}").split(",")
    domains = [d.strip() for d in domains]  # clean up whitespace if any
    
    # Reset bool_check for this row
    bool_check = [0]*8
    
    # Check which A-fields are present
    for i in range(8):
        if f'A{i+1}' in domains:
            bool_check[i] = 1
    
    # Construct the filtered list
    filtered = [f'A{i+1}' for i in range(8) if bool_check[i] == 1]
    
    # Assign the processed list back to the DataFrame
    course.at[idx, '通識領域'] = filtered



In [186]:
# for _, c in course.iterrows():
#     # if c['通識領域'] conatins >= 2 elements, print the row
#     if len(c['通識領域']) >= 2:
#         print(c['通識領域'])

In [187]:
# see how many rows left
print(course.shape)

(8107, 13)


In [188]:
# save the data to a csv file
course.to_csv('course-v1.csv', index=False)

---

##### how to embedding data
- only 課程概述 + 課程名稱?
- 還是要加課程上課日期, 課程目標, etc


##### multi label firtering
- 我要找文字探勘和 sentence embedding 相關的課程， "而且想要禮拜三早上或是禮拜五的"
- 前面可以用 RAG + LLM 處理，後面那部分如何？

##### 
'課程名稱', '課程概述', '課程目標', '課程要求' 加起來超過 512 個 token 怎麼辦？

- 叫 LLM 抓重點？
- ~~休學~~
- 把「該門課程依照字數限制拆成兩個 ( or 多個 ) item」，如有 1700 token 的話，拆成四個 item ？